In [1]:
raw_data = sc.textFile('hdfs://master:54310/raw_data')

In [2]:
import json
data = raw_data.map(json.loads)

In [3]:
unique = data.groupBy(lambda x: x['Uri']). \
    map(lambda g: list(g[1])[0])

In [4]:
from nltk.tokenize import regexp_tokenize

def split_text(text: str):
    return regexp_tokenize(text, '''[\w']+''')

In [5]:
import re
def filter_variable_names(words: list):
    return [word for word in words if '_' not in word]

def delete_non_word_chars(text: str):
    temp = text#replace_urls(text)
    temp = temp.replace('ё', 'е')
    temp = re.sub(r'(&[a-z0-9]*;)', ' ', temp) # & encoded symbols
    temp = re.sub(r'(\W|\d)+', ' ', temp) # non word or digit
    temp = re.sub(r'\s+', ' ', temp) # 2+ spaces
    return temp.strip()

In [6]:
from nltk.corpus import stopwords

russian_stops = stopwords.words('russian')
english_stops = stopwords.words('english')

def filter_stopwords(words: list):
    return [word for word in words \
           if word not in russian_stops \
           and word not in english_stops]

In [7]:
def filter_words_with_repeatable_letters(words: list):
    return [word for word in words if not re.match('(.)\\1{2}', word)]
    
def is_language_usual(word: str):
    length = len(word)
    is_eng = re.match('[a-z]', word)
    return length > 2 and ((not is_eng and length < 25) or (is_eng and length < 15))

def filter_words_with_unusual_by_language_length(words: list):
    return [word for word in words if is_language_usual(word)]

In [8]:
def tokenize_text(text: str):
    text = text.lower()
    text = delete_non_word_chars(text)
    tokens = split_text(text)
    tokens = filter_variable_names(tokens)
    tokens = filter_stopwords(tokens)
    tokens = filter_words_with_repeatable_letters(tokens)
    tokens = filter_words_with_unusual_by_language_length(tokens)
    return tokens

In [9]:
def normalize_label(label: str):
    temp = label.lower()
    temp = temp.replace('ё', 'е')
    temp = split_text(temp)
    return ' '.join(temp)

label_bl = {'лог компании', 'рная дыра', 'пиарюсь'}
def filter_bl_labels(labels: list):
    return [l for l in labels if
            all([l_bl not in l for l_bl in label_bl])]

def normalize_labels(labels: list):    
    return [normalize_label(l_i) for l_i in filter_bl_labels(labels)]    

In [10]:
def transform_data(data: dict):
    res = {}
    res['Labels'] = normalize_labels(data['Hubs'] + data['Tags'])
    res['Id'] = data['Number']
    res['Features'] = tokenize_text(data['Text'])
    return res

In [12]:
unique.map(lambda x: transform_data(x)). \
    filter(lambda x: len(x['Features']) > 0 and len(x['Labels']) > 2). \
    map(lambda x: json.dumps(x)). \
    repartition(6). \
    saveAsTextFile('hdfs://master:54310/unique_not_stemmed')